In [ ]:
import os
import openai
import csv
openai.api_key = "" #Enter your API key here
openai.Engine.list()

In [ ]:
model_name = "curie"
openai.Engine.retrieve(model_name)

In [ ]:
#---------- This function gets the dataset and returns a prompt

def data_to_prompt(task_description, train_file_address, test_text):
    prompt = ""
    prompt += task_description + '\n'
    
    with open(train_file_address) as train_file:
        train_data = csv.reader(train_file, delimiter='\t')
    
        for row in train_data:
            sample_text = row[0]
            sample_label = row[1]
        
            prompt += "Text:" + sample_text + '\n'
            prompt += "Label:" + sample_label + '\n'
            prompt += "###\n"
        
        prompt += "Text:" + test_text + '\n'
        prompt += "Label:"
    
    return prompt

In [ ]:
train_address = 'Data\\MedSTS-train-20.tsv'
test_address = 'Data\\MedSTS-test-full.tsv'

max_label_token = 10

test_scores = []
predictions = []

task_description = "Specify the semantic similarity between the sentences"

total_samples = 0
correct_predictions = 0

with open(test_address) as test_file:
    test_data = csv.reader(test_file, delimiter='\t')
    
    for row in test_data:
        sample_text = row[0]
        sample_label = row[1]
        
        prompt = data_to_prompt(task_description, train_address, sample_text)
        response = openai.Completion.create(engine=model_name, prompt=prompt, temperature=0, max_tokens=max_label_token, top_p=1, frequency_penalty=0, presence_penalty=0, stop=["\n"])
        print(prompt)
        print("*************")
        print(response)
        choices = response["choices"]
        predicted_label = choices[0]
        predicted_label = predicted_label["text"]
        if(len(predicted_label) > 0):
            if(predicted_label[0] == ' '):
                predicted_label = predicted_label[1:]
        print('Test text:', sample_text, ' || Test label:', sample_label, ' || Predicted label:', predicted_label)
        print("-----------------------------------------------------------------------------------")
        
        
        total_samples += 1
        if(sample_label.lower() == predicted_label.lower()):
            correct_predictions += 1

accuracy = correct_predictions / total_samples        
print('Total samples:', total_samples)
print('Correct predictions:', correct_predictions)
print('Accuracy:', accuracy)